In [ ]:
import numpy as np
from tinydb import TinyDB 
from tinydb import Query 
from tinydb import where

import pickle
import pandas as pd
from tqdm import tqdm

In [ ]:
db = TinyDB('nav_db.json')
Q = Query()

In [ ]:
rosPicklePath = "../name.ros_pickle"
with open(rosPicklePath,"rb") as f:
    demoPickle = pickle.load(f)
demoPickle.keys()
GNSSInfor = demoPickle['GNSS']
GNSSInfor[500]

In [ ]:
GNSSinforpart = []
for item in GNSSInfor:
    if "datarawx_week" in item:
        if type(item["datarawx_week"]) == list:
            week = item["datarawx_week"][0]
            gpstime = item["datarawx_rcvtow"][0]
        if type(item["datarawx_week"]) == int:
            week = item["datarawx_week"]
            gpstime = item["datarawx_rcvtow"]
    else:
        week = None
        gpstime = None
    partItem = {
        "UTCTime":item["stamp_sec"],
        "Week":week,
        "GPSTime":gpstime,
        "Latitude":item["datahpposllh_lat"]*1e-8    if "datahpposllh_lat" in item else None,
        "Longitude":item["datahpposllh_lon"]*1e-8   if "datahpposllh_lon" in item else None,
        "H-Ell":item["datahpposllh_hmsl"]*1e-4      if "datahpposllh_hmsl" in item else None,
        "VelBdyX":None,        "VelBdyY":None,        "VelBdyZ":None,
        "AccBdyX":None,        "AccBdyY":None,        "AccBdyZ":None,
        "Roll":None,        "Pitch":None,        "Heading":None,        "Q":None,
    }
    GNSSinforpart.append(partItem)
df = pd.DataFrame(GNSSinforpart)

In [ ]:
insertGroup = []
for item in df.to_dict(orient='records'):
    insertGroup.append(
        {"UTCTime":item["UTCTime"],"Week":item["Week"],"GPSTime":item["GPSTime"],
        "GT":{
            "Latitude":item["Latitude"],
            "Longitude":item["Longitude"],
            "H-Ell":item["H-Ell"],
            "VelBdyX":item["VelBdyX"],
            "VelBdyY":item["VelBdyY"],
            "VelBdyZ":item["VelBdyZ"],
            "AccBdyX":item["AccBdyX"],
            "AccBdyY":item["AccBdyY"],
            "AccBdyZ":item["AccBdyZ"],
            "Roll":item["Roll"],
            "Pitch":item["Pitch"],
            "Heading":item["Heading"],
            "Q":item["Q"]
            }
        }
    )
_ = db.insert_multiple(insertGroup)

In [ ]:
satEAList = []

satInforList= []

for timeIndex in tqdm(range(len(GNSSInfor))):
    Elevlist = GNSSInfor[timeIndex]['datasat_elev']
    AzimList = GNSSInfor[timeIndex]['datasat_azim']
    if type(Elevlist) != list: Elevlist = [Elevlist]
    if type(AzimList) != list: AzimList = [AzimList]
    GPS_EA = {}
    for i in range(len(Elevlist)):
        GPS_EA[str(i+1)] = (Elevlist[i],AzimList[i])
    satEAList.append(
        (
            {'EA':GPS_EA},
            where('UTCTime')==GNSSInfor[timeIndex]['stamp_sec']
        )
    )
        
_ = db.update_multiple(satEAList)

In [ ]:
satObsList = []
satObDict = {}


for item in GNSSInfor:
    if "datarawx_svid" in item:
        for i in range(len(item['datarawx_svid'])):
            obsDict = {
                "prn":item['datarawx_svid'][i],
                "pr":item['datarawx_prmes'][i],
                "cp":item['datarawx_cpmes'][i],
                "doppler":item['datarawx_domes'][i],
                "cn0":item['datarawx_cno'][i],
                "elev":item['datasat_elev'][i],
                "azim":item['datasat_azim'][i]
            }
            if float(item['stamp_sec']) in satObDict:
                satObDict[float(item['stamp_sec'])].append(obsDict)
            else:
                satObDict[float(item['stamp_sec'])] = [obsDict]

for k,v in satObDict.items():
    satObsList.append(
        (
            {"Obs":v},
            where('UTCTime')==k
        )
    )
    # break
_ = db.update_multiple(satObsList)

In [ ]:
def find_index(arr, value):
    indices = np.where(arr == value)[0]  # 获取所有匹配的索引，这里假设数组是一维的
    if indices.size > 0:
        return indices[0]  # 返回第一个匹配的索引
    else:
        return None  # 如果没有找到匹配的值，返回 None
def compute_mean_of_less_than(arr1, arr2, value):
    # Step 1: 找出第一个数组中小于给定值的所有索引
    mask = arr1 < value
    indices = np.where(mask)[0]
    #print(indices)
    # Step 2: 使用索引读取第二个数组的对应元素，并计算平均值
    selected_values = arr2[indices]
    mean_value = np.mean(selected_values)
    return mean_value
def cal_GNSS_feature(CN0_sat,ELEV_sat):
    VSR_cn0_feature = np.sum(CN0_sat>30)
    VSR_ele_feature= np.sum(ELEV_sat<45)
    mean_cn0 = np.mean(CN0_sat)
    std_cn0 = np.std(CN0_sat)
    low_qu_cn0,up_qu_cn0 = np.percentile(CN0_sat, (25, 75), interpolation='midpoint')
    mean_elev = np.mean(ELEV_sat)
    std_elev = np.std(ELEV_sat)
    cn0_elev= compute_mean_of_less_than(ELEV_sat,CN0_sat,30)
    return VSR_cn0_feature,VSR_ele_feature,mean_cn0,std_cn0,low_qu_cn0,up_qu_cn0,mean_elev,std_elev,cn0_elev
def cal_MP(PR_sat,Previous_PR_sat,PRN_sat,Previous_PRN_sat,Doppler_sat):
    MP_sat = np.zeros(len(PR_sat))
    for MP_index in range(len(PR_sat)):
        MP_pre_index = find_index(Previous_PRN_sat,PRN_sat[MP_index])
        if MP_pre_index is not None:
            # print("cal_MP--MP_pre_index:",MP_pre_index)
            # print("cal_MP--PR_sat:",PR_sat[MP_index])
            # print("cal_MP--Previous_PR:",Previous_PR_sat[MP_pre_index])
            MP = PR_sat[MP_index] - Previous_PR_sat[MP_pre_index] - (Doppler_sat[MP_index] * 0.1903)
            # print("MP:",MP)
            MP_sat[MP_index] = MP
        else:
            MP_sat[MP_index] = None
    MP_sat = np.nan_to_num(MP_sat, nan=np.nanmean(MP_sat))
    # print(MP_sat)
    return MP_sat
def cal_MP_feature(MP_sat,ELEV_sat):
    #print('cal_MP_feature--MP_sat:',MP_sat)
    #print('cal_MP_feature--ELEV_sat:',ELEV_sat)
    mean_MP = np.mean(MP_sat)
    range_MP = np.max(MP_sat) - np.min(MP_sat)
    mean_MP_elev = compute_mean_of_less_than(ELEV_sat,MP_sat,60)
    return mean_MP,range_MP,mean_MP_elev
def deg2rad(deg):
    return np.deg2rad(deg)
def calculate_cofactor_matrix(H):
    # 设置mpmath的精度（十进制位数）
    # mpmath.mp.dps = precision
    
    # 转置矩阵 H^T
    #H_transpose = mpmath.matrix(H).transpose()
    H_transpose = H.transpose()
    
    # 计算 H^T * H
    #Ht_H = H_transpose * mpmath.matrix(H)
    Ht_H = H_transpose*H
    
    # 计算逆矩阵 (H^T * H)^-1
    try:
        Pi_n = Ht_H**-1
    except ZeroDivisionError:
        raise ValueError("Matrix is singular and cannot be inverted.")
    
    return Pi_n
def cal_DOP(ELEV_sat, AZIM_sat):
    ELEV_sat = np.asarray(ELEV_sat, dtype=float)
    AZIM_sat = np.asarray(AZIM_sat, dtype=float)
    if ELEV_sat.shape != AZIM_sat.shape:
        raise ValueError("ELEV_sat 与 AZIM_sat 长度不一致")
    N = len(ELEV_sat)

    # 至少 4 颗卫星才可计算完整 DOP
    if N < 4:
        raise ValueError("卫星数不足 4，无法计算完整 DOP（GDOP/VDOP/PDOP/TDOP）。")

    # 角度转弧度
    AZ = np.deg2rad(AZIM_sat)
    EL = np.deg2rad(ELEV_sat)

    # 列0: -cos(AZ)*cos(EL)  -> North (取反)
    # 列1: -sin(AZ)*cos(EL)  -> East  (取反)
    # 列2: -sin(EL)          -> Up    (取反)
    # 列3: 1                 -> clock bias
    G = np.column_stack([
        -np.cos(AZ) * np.cos(EL),
        -np.sin(AZ) * np.cos(EL),
        -np.sin(EL),
         np.ones(N)
    ])

    GTG = G.T @ G
    # 优先尝试 inv，若奇异则用 pinv
    try:
        H = np.linalg.inv(GTG)
    except np.linalg.LinAlgError:
        H = np.linalg.pinv(GTG)

    NDOP = np.sqrt(H[0,0])
    EDOP = np.sqrt(H[1,1])
    VDOP = np.sqrt(H[2,2])
    TDOP = np.sqrt(H[3,3])
    HDOP = np.sqrt(H[0,0] + H[1,1])
    PDOP = np.sqrt(H[0,0] + H[1,1] + H[2,2])
    GDOP = np.sqrt(H[0,0] + H[1,1] + H[2,2] + H[3,3])

    return GDOP, VDOP

In [ ]:
'''
prn:卫星号
pr:伪距
cp:载波相位
doppler:多普勒频移
cn0：载噪比
elev：高度角
azim：方位角
'''
'''
需要提取的特征有：
1、可见卫星数量 VSR_feature
2、载噪比>30的可见卫星数量VSR_cn0_feature
3、高度角<45的可见卫星数量VSR_ele_feature
4、平均载噪比 mean_cn0
5、载噪比的标准差std_cn0
6、uq_载噪比(upper quartile 四分位数)up_qu_cn0
7、lq_载噪比(lower quartile 四分位数)low_qu_cn0
8、载噪比（elev<30）?cn0_elev<30
9、mean_MP
10、range_MP
11、mean_MP（elev<60）
12、平均高度角 mean_elev
13、高度角标准差 std_elev
14、GDOP
15、VDOP
做一个新的Json数据库，或者直接插入到这个里面
'''

Q = Query()
allUTCtime = db.search(Q.UTCTime.exists())
allUTCtime = [item['UTCTime'] for item in allUTCtime if 'Obs' in item]

batchUpdate = []
for UTCTimeindex in tqdm(range(len(allUTCtime))):
    #print('UTCTimeindex:',UTCTimeindex)
    UTCTime = allUTCtime[UTCTimeindex]
    doc = db.search(Q.UTCTime == UTCTime)
    UTCTime_Obs = doc[0]["Obs"]
    VSR_feature = len(UTCTime_Obs)
    CN0_sat = np.zeros(len(UTCTime_Obs))
    ELEV_sat = np.zeros(len(UTCTime_Obs))
    
    AZIM_sat= np.zeros(len(UTCTime_Obs))
    Doppler_sat= np.zeros(len(UTCTime_Obs))
    PR_sat = np.zeros(len(UTCTime_Obs))
    PRN_sat = np.zeros(len(UTCTime_Obs))
    for Obs_index in range(len(UTCTime_Obs)):
        CN0_sat[Obs_index] = UTCTime_Obs[Obs_index]['cn0']
        ELEV_sat[Obs_index]= UTCTime_Obs[Obs_index]['elev']
        AZIM_sat[Obs_index]= UTCTime_Obs[Obs_index]['azim']
        Doppler_sat[Obs_index]= UTCTime_Obs[Obs_index]['doppler']
        PR_sat[Obs_index]= UTCTime_Obs[Obs_index]['pr']
        PRN_sat[Obs_index]= UTCTime_Obs[Obs_index]['prn']
    VSR_cn0_feature,VSR_ele_feature,mean_cn0,std_cn0,low_qu_cn0,up_qu_cn0,mean_elev,std_elev,cn0_elev = cal_GNSS_feature(CN0_sat,ELEV_sat)
    if ELEV_sat.shape[0] < 4:
        GDOP,VDOP = -1,-1
    else:
        # print(ELEV_sat.shape[0])
        GDOP,VDOP = cal_DOP(ELEV_sat,AZIM_sat)
    if UTCTimeindex ==0:
        Previous_PR_sat= PR_sat
        Previous_PRN_sat= PRN_sat
        MP_sat= np.zeros(len(UTCTime_Obs))
        mean_MP = 0
        range_MP = 0
        mean_MP_elev = 0
    else:
        MP_sat = cal_MP(PR_sat,Previous_PR_sat,PRN_sat,Previous_PRN_sat,Doppler_sat)
        mean_MP,range_MP,mean_MP_elev = cal_MP_feature(MP_sat,ELEV_sat)
        Previous_PR_sat  = PR_sat
        Previous_PRN_sat = PRN_sat
    dict_gnss_feature = {   "VSR_feature":int(VSR_feature), "VSR_cn0_feature":int(VSR_cn0_feature),"VSR_ele_feature":int(VSR_ele_feature),
                            "mean_cn0":round(float(mean_cn0),5),"std_cn0":round(float(std_cn0),5),"up_qu_cn0":round(float(up_qu_cn0),5),
                            "low_qu_cn0":round(float(low_qu_cn0),5),"cn0_elev":round(float(cn0_elev),5),
                            "mean_elev":round(float(mean_elev),5),"std_elev":round(float(std_elev),5),
                            "GDOP":round(float(GDOP),5),"VDOP":round(float(VDOP),5),"mean_MP":round(float(mean_MP),5),
                            "range_MP":round(float(range_MP),5),"mean_MP_elev":round(float(mean_MP_elev),5)}
    
    batchUpdate.append(
        (
            {"dict_gnss_feature":dict_gnss_feature},
            where('UTCTime')==UTCTime
        )
    )
_ = db.update_multiple(batchUpdate)